In [1]:
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import cm
from scipy import stats
from collections import deque
from scipy.optimize import minimize
from scipy.linalg import norm, inv, det
import time
import seaborn as sns
from multiprocess import Pool, cpu_count
sns.set_theme()

# Functions

In [2]:
class multivariate_exponential_hawkes(object):
    """
    Multivariate Hawkes process with exponential kernel. No events nor initial conditions considered.
    """

    def __init__(self, mu, alpha, beta, max_jumps=None, max_time=None, burn_in=0.0):
        """

        Parameters
        ----------
        mu : array_like
            Baseline intensity vector. mu.shape[0] must coincide with shapes for alpha and beta.
        alpha : array_like
            Interaction factors matrix. Must be a square array with alpha.shape[0] coinciding with mu and beta.
        beta : array_like
            Decay factor matrix. Must be either an array. When corresponding to decay for each process i, it must
            be of shape (number_of_process, 1), or a square array. beta.shape[0] must coincide with mu and alpha.
        max_jumps : float, optional
            Maximal number of jumps. The default is None.
        max_time : float, optional
            Maximal time horizon. The default is None.
        burn_in : float, optional
            Burn-in period for stationary simulation.

        Attributes
        ----------
        nb_processes : int
            Number of dimensions.
        timestamps : list of tuple (float, int)
            List of simulated events and their marks.
        intensity_jumps : array of float
            Array containing all intensities at each jump. It includes the baseline intensities mu.
        simulated : bool
            Parameter that marks if a process has been already been simulated,
            or if its event times have been initialized.

        """

        # We must begin by verifying that the process is a point process. In other words, that the number of
        # points in any bounded interval is a.s. finite. For this, we have to verify that the spectral radius of
        # the matrix alpha/beta (term by term) is <1.

        beta_radius = np.copy(beta)
        beta_radius[beta_radius == 0] = 1
        self.spectral_radius = np.max(np.abs(np.linalg.eig(np.abs(alpha))[0]))

        if self.spectral_radius >= 1:
            # raise ValueError("Spectral radius is %s, which makes the process unstable." % (spectral_radius))
            warnings.warn("Spectral radius is %s, which may make the process unstable." % (self.spectral_radius),RuntimeWarning)
        self.mu = mu.reshape((alpha.shape[0], 1))
        self.alpha = alpha
        if beta.shape[1] != alpha.shape[0]: # Initialisation of beta if constant by process
            self.beta = np.repeat(beta, alpha.shape[0], axis=1)
        else:
            self.beta = beta
        self.max_jumps = max_jumps
        self.max_time = max_time

        self.before_origin_time = 0.0
        self.aux_before = 0.0 * self.alpha
        self.at_origin_intensity = self.mu

        self.burn_in = -np.abs(burn_in)

        self.nb_processes = self.mu.shape[0]
        self.count = np.zeros(self.nb_processes, dtype=int)

        self.timestamps = [(burn_in, 0)]
        self.intensity_jumps = np.copy(mu)

        self.simulated = False

    def simulate(self):
        """
        Auxiliary function to check if already simulated and, if not, which simulation to launch.

        Simulation follows Ogata's adapted thinning algorithm. Upper bound obtained by the positive-part process.

        Works with both self-exciting and self-regulating processes.

        To launch simulation either self.max_jumps or self.max_time must be other than None, so the algorithm knows when to stop.
        """
        if not self.simulated:
            if self.max_jumps is not None and self.max_time is None:
                self.simulate_jumps()
            elif self.max_time is not None and self.max_jumps is None:
                if self.spectral_radius >= 1.0:
                    raise ValueError("Spectral radius is %s, simulation with max_time may not end. Prefer providing max_jumps." % (self.spectral_radius))
                self.simulate_time()
            else:
                print("Either max_jumps or max_time must be given.")
            self.simulated = True
            #self.complete_times = self.timestamps
            self.timestamps = [(0.0, 0)] + [(t, m) for t,m in self.timestamps if t > 0.0]

        else:
            print("Process already simulated")

    def simulate_jumps(self):
        """
        Simulation is done until the maximal number of jumps (self.max_jumps) is attained.
        """
        flag = 0
        t = self.burn_in

        auxiliary_alpha = np.where(self.alpha > 0, self.alpha, 0)
        auxiliary_ij = np.zeros((self.nb_processes, self.nb_processes))
        auxiliary_intensity = np.copy(self.mu)

        ij_intensity = np.zeros((self.nb_processes, self.nb_processes))

        while flag < self.max_jumps:

            upper_intensity = np.sum(auxiliary_intensity)

            previous_t = t
            t += np.random.exponential(1 / upper_intensity)
            if t > 0.0 > previous_t:
                self.before_origin_time = previous_t
                self.aux_before = np.multiply(ij_intensity, np.exp(-self.beta * (0 - previous_t)))
                self.at_origin_intensity = self.mu + np.sum(self.aux_before, axis=1, keepdims=True)

            # ij_intensity = np.multiply(ij_intensity, np.exp(-self.beta * (t - self.timestamps[-1][0])))
            ij_intensity = np.multiply(ij_intensity, np.exp(-self.beta * (t - previous_t)))
            candidate_intensities = self.mu + np.sum(ij_intensity, axis=1, keepdims=True)
            pos_candidate = np.maximum(candidate_intensities, 0) / upper_intensity
            type_event = np.random.multinomial(1, np.concatenate((pos_candidate.squeeze(axis=1), np.array([0.0])))).argmax()
            if type_event < self.nb_processes:
                self.timestamps += [(t, type_event + 1)]
                ij_intensity[:, type_event] += self.alpha[:, type_event] * self.beta[:, type_event]
                self.intensity_jumps = np.c_[
                    self.intensity_jumps, self.mu + np.sum(ij_intensity, axis=1, keepdims=True)]

                auxiliary_ij = np.multiply(auxiliary_ij, np.exp(-self.beta * (t - self.timestamps[-2][0])))
                auxiliary_ij[:, type_event] += auxiliary_alpha[:, type_event] * self.beta[:, type_event]
                auxiliary_intensity = self.mu + np.sum(auxiliary_ij, axis=1, keepdims=True)

                flag += 1

                self.count[type_event] += 1

        self.max_time = self.timestamps[-1][0]
        # Important to add the max_time for plotting and being consistent.
        self.timestamps += [(self.max_time, 0)]
        #self.timestamps = [(t,m) for t,m in self.timestamps()]

    def simulate_time(self):
        """
        Simulation is done for a window [0, T] (T = self.max_time) is attained.
        """
        t = self.burn_in
        flag = t < self.max_time

        auxiliary_alpha = np.where(self.alpha > 0, self.alpha, 0)
        auxiliary_ij = np.zeros((self.nb_processes, self.nb_processes))
        auxiliary_intensity = np.copy(self.mu)

        ij_intensity = np.zeros((self.nb_processes, self.nb_processes))

        while flag:

            upper_intensity = np.sum(auxiliary_intensity)

            previous_t = t
            t += np.random.exponential(1 / upper_intensity)
            if t > 0.0 > previous_t:
                self.before_origin_time = previous_t
                self.aux_before = np.multiply(ij_intensity, np.exp(-self.beta * (0 - previous_t)))
                self.at_origin_intensity = self.mu + np.sum(self.aux_before, axis=1, keepdims=True)
            #print("cand", t, upper_intensity)

            # ij_intensity = np.multiply(ij_intensity, np.exp(-self.beta * (t - self.timestamps[-1][0])))
            ij_intensity = np.multiply(ij_intensity, np.exp(-self.beta * (t - previous_t)))
            candidate_intensities = self.mu + np.sum(ij_intensity, axis=1, keepdims=True)
            pos_candidate = np.maximum(candidate_intensities, 0) / upper_intensity
            #print((pos_candidate.squeeze(axis=1), pos_candidate.squeeze(), np.array([0.0])))
            type_event = np.random.multinomial(1,
                                               np.concatenate((pos_candidate.squeeze(axis=1), np.array([0.0])))).argmax()
            flag = t < self.max_time
            if type_event < self.nb_processes and flag:
                self.timestamps += [(t, type_event + 1)]
                ij_intensity[:, type_event] += self.alpha[:, type_event] * self.beta[:, type_event]
                self.intensity_jumps = np.c_[
                    self.intensity_jumps, self.mu + np.sum(ij_intensity, axis=1, keepdims=True)]

                auxiliary_ij = np.multiply(auxiliary_ij, np.exp(-self.beta * (t - self.timestamps[-2][0])))
                auxiliary_ij[:, type_event] += auxiliary_alpha[:, type_event] * self.beta[:, type_event]
                auxiliary_intensity = self.mu + np.sum(auxiliary_ij, axis=1, keepdims=True)

                self.count[type_event] += 1

        self.timestamps += [(self.max_time, 0)]


    def plot_intensity(self, ax=None, plot_N=True, where=10):
        """
        Plot intensity function. If plot_N is True, plots also step functions N^i([0,t]).
        The parameter ax allows to plot the intensity function in a previously created plot.

        Parameters
        ----------
        ax : array of Axes, optional.
            If None, method will generate own figure.
            Otherwise, will use given axes. Must be array of shape (2,K) if plot_N = True, or (K,) if plot_N = False
        plot_N : bool, optional.
            Whether we plot the step function N^i or not.
        """

        if not self.simulated:
            print("Simulate first")

        else:
            plt.rcParams['axes.grid'] = True
            if plot_N:
                jumps_plot = [[0] for i in range(self.nb_processes)]
                if ax is None:
                    fig, ax = plt.subplots(2, self.nb_processes, sharex=True)
                elif isinstance(ax[0,0], matplotlib.axes.Axes):
                    pass
                else:
                    return "ax is the wrong shape. It should be (2, number of processes+1)"
                ax1 = ax[0]
                ax2 = ax[1]
                if self.nb_processes == 1:
                    ax1 = [ax1]
                    ax2 = [ax2]
            else:
                if ax is None:
                    fig, ax1 = plt.subplots(1, self.nb_processes)
                elif isinstance(ax, matplotlib.axes.Axes) or isinstance(ax, np.ndarray):
                    ax1 = ax
                else:
                    return "ax is the wrong shape. It should be (number of processes+1,)"
                if self.nb_processes == 1:
                    ax1 = [ax1]


            ij_intensity = self.aux_before

            step = 100

            if self.before_origin_time < 0.0:
                func = lambda x: self.mu + np.matmul(
                    np.multiply(ij_intensity, np.exp(-self.beta * (x))),
                    np.ones((self.nb_processes, 1)))

                # On enregistre la division de temps et les sauts
                interval_t = np.linspace(0.0, self.timestamps[1][0], step)
                times = interval_t.tolist()
                intensities = np.array(list(map(func, interval_t))).squeeze(axis=-1).T

            else:
                times = [0, self.timestamps[1][0]]
                intensities = np.array([[self.mu[i, 0], self.mu[i, 0]] for i in range(self.nb_processes)])

            # print("here", self.timestamps[len(self.timestamps)])
            #print("where",len(self.timestamps[1:where]), where)
            for i in range(1, len(self.timestamps[1:where])):
                # On commence par mettre à jour la matrice lambda^{ij}
                ij_intensity = np.multiply(ij_intensity,
                                           np.exp(-self.beta * (self.timestamps[i][0] - self.timestamps[i - 1][0])))
                # On enregistre le saut d'intensité de l'évenement, pour son type.
                ij_intensity[:, self.timestamps[i][1]-1] += self.alpha[:, self.timestamps[i][1]-1] * self.beta[:, self.timestamps[i][1]-1]

                # On définit la fonction à tracer entre T_n et T_{n+1}
                func = lambda x: self.mu + np.matmul(
                    np.multiply(ij_intensity, np.exp(-self.beta * (x - self.timestamps[i][0]))),
                                np.ones((self.nb_processes, 1)))

                # On enregistre la division de temps et les sauts
                interval_t = np.linspace(self.timestamps[i][0], self.timestamps[i + 1][0], step)
                times += interval_t.tolist()
                intensities = np.concatenate((intensities, np.array(list(map(func, interval_t))).squeeze(axis=-1).T ), axis=1)
                if plot_N:
                    jumps_plot[self.timestamps[i][1]-1] += [self.timestamps[i][0] for t in range(2)]

            for i in range(self.nb_processes):
                ax1[i].plot(times, intensities[i], label="Underlying intensity", c="#1f77b4", linestyle="--")
                ax1[i].plot(times, np.maximum(intensities[i], 0), label="Conditional intensity", c='r')
                # ax1[i].plot([i for i,j in self.timestamps[:-1]], self.intensity_jumps[i,:], c='k', alpha=0.5)

            ax1[0].legend()

            if plot_N:
                for i in range(self.nb_processes):
                    jumps_plot[i] += [times[-1]]
                    ax2[i].plot(jumps_plot[i], [t for t in range(len(jumps_plot[i])//2) for j in range(2)], c="r", label="Process #%s"%(i+1), zorder=10)
                    # ax2[i].set_ylim(ax2[i].get_ylim())
                    for j in range(self.nb_processes):
                        if j != i:
                            ax2[j].plot(jumps_plot[i], [t for t in range(len(jumps_plot[i])//2) for j in range(2)], c="#1f77b4", alpha=0.5, zorder=5)

                    ax2[i].legend()



    def plot_heatmap(self, ax=None):
        """
        This function allows to observe the heatmap where each cell {ij} corresponds to the value {alpha/beta} from that interaction

        Parameters
        ----------
        ax : .axes.Axes, optional.
            If None, method will generate own ax.
            Otherwise, will use given ax.
        """
        import seaborn as sns

        if ax is None:
            fig, ax = plt.subplots()
        else:
            ax = ax
        beta_heat = np.copy(self.beta)
        beta_heat[beta_heat == 0] = 1
        heat_matrix = self.alpha

        hex_list = ['#FF3333', '#FFFFFF', '#33FF49']

        ax = sns.heatmap(heat_matrix, cmap=get_continuous_cmap(hex_list), center=0, ax=ax, annot=True)


In [3]:
import finufft
def fast_multi_periodogram(K, tList, max_time, precision=1e-9):
    
    dim = int(np.max(np.array(tList)[:, 1]))
    dimensional_times = [[t for t,i in tList if i == j] for j in range(1, dim+1)]
    
    # put K for w=0
    aux = np.array([finufft.nufft1d1(2*np.pi*np.array(x)/max_time, np.ones(len(x)) + 0j, n_modes = 2*K+1, isign=-1, eps=1e-9)[K+1:] for x in dimensional_times]) 
    aux = (aux.T)[:,:, np.newaxis]
    aux = (aux @ np.transpose(np.conj(aux), axes=(0,2,1))) / max_time
    
    return aux

def ei(size, index):
    e = np.zeros((size))
    e[index] = 1.0
    return e

In [4]:
def grad_spectral_w_mask(theta, w, periodogramw, mask): # The mask allows to set values to 0.
    mu0, alpha0, beta0, noise0 = theta

    dim = mu0.shape[0]
    a = inv(np.identity(dim) - alpha0)

    mean_matrix = np.identity(dim) * (a @ mu0)

    fourier_matrix = alpha0 * beta0 / (beta0 + 2j * np.pi * w)
    spectral_matrix = inv(np.identity(dim) - fourier_matrix)

    f_theta_unnoised = (spectral_matrix) @ mean_matrix @ np.conj(spectral_matrix.T)
    f_theta = f_theta_unnoised + noise0 * np.identity(dim)
    f_inv = inv(f_theta)

    ll = np.log(det(f_theta)) + np.trace(f_inv @ periodogramw)

    #dmu = np.zeros((dim, dim, dim), dtype=np.complex128)
    #dalpha = np.zeros((dim, dim, dim, dim), dtype=np.complex128)
    #dbeta = np.zeros((dim, dim, dim), dtype=np.complex128)
    aux_dbeta = alpha0 * (2j * np.pi * w) * np.repeat(1 / (beta0 + 2j * np.pi * w) ** 2, dim, axis=1)

    dmu = a @ np.array([ei(((dim, dim)), i) for i in range(dim)]) * np.array([np.identity(dim)] * dim)
    dbeta = aux_dbeta * np.array([ei((dim, dim), i) for i in range(dim)])

    dij = mask * np.array([[ei(dim, i)[:, np.newaxis] * ei(dim, j)[np.newaxis, :] for j in range(dim)] for i in range(dim)])
    dalpha_cent = a @ dij @ a @ mu0
    dalpha_cent = dalpha_cent * np.array([[np.identity(dim)] * dim] * dim)
    dalpha_bord = dij * beta0 / (beta0 + 2j * np.pi * w)

    dmu = spectral_matrix @ dmu @ np.conj(spectral_matrix.T)
    dalpha = (spectral_matrix @ dalpha_bord @ f_theta_unnoised) + (
                f_theta_unnoised @ np.transpose(np.conj(dalpha_bord), axes=(0, 1, 3, 2)) @ np.conj(spectral_matrix.T))
    dalpha += spectral_matrix @ dalpha_cent @ np.conj(spectral_matrix.T)
    dbeta = (spectral_matrix @ dbeta @ f_theta_unnoised) + (
                f_theta_unnoised @ np.transpose(np.conj(dbeta), axes=(0, 2, 1)) @ np.conj(spectral_matrix.T))
    dnoise = np.identity(dim)
    aux_det = f_inv.T

    aux_trace_mu = (aux_det.T) @ dmu @ (aux_det.T)
    aux_trace_alpha = (aux_det.T) @ dalpha @ (aux_det.T)
    aux_trace_beta = (aux_det.T) @ dbeta @ (aux_det.T)
    aux_trace_noise = (aux_det.T) @ dnoise @ (aux_det.T)

    dmu = np.sum(aux_det * dmu, axis=(1,2)) - np.sum((periodogramw.T) * aux_trace_mu,
                                                                        axis=(1,2))
    dalpha = np.sum(aux_det * dalpha, axis=(2, 3)) - np.sum((periodogramw.T) * aux_trace_alpha, axis=(2, 3))
    dbeta = np.sum(aux_det * dbeta, axis=(1,2)) - np.sum((periodogramw.T) * aux_trace_beta,
                                                                            axis=(1,2))
    dnoise = np.sum(aux_det * dnoise) - np.sum((periodogramw.T) * aux_trace_noise)

    #print(alpha0, dalpha)

    grad_final = np.concatenate((dmu.real.ravel(), dalpha.real.ravel(), dbeta.real.ravel(), np.array([dnoise.real])))

    return np.concatenate((np.array([ll.real]), grad_final))

In [5]:
def literal_dalpha(mu, alpha, beta, noise, w, periodogramw):

    f = (mu/((1-alpha))) * (1 + ((beta**2) * alpha * (2-alpha))/((beta * (1-alpha))**2 + (2 * np.pi * w)**2)) + noise
    
    res = (mu/((1-alpha)**2)) * (1 + ((beta**2) * alpha * (2-alpha))/((beta * (1-alpha))**2 + (2 * np.pi * w)**2))
    res += 2 * mu * (beta**2) * ((beta)**2 + (2 * np.pi * w)**2) / (((beta * (1-alpha))**2 + (2 * np.pi * w)**2)**2)

    res = res / f - periodogramw * res / (f**2)
    
    return res

In [6]:
def uni_grad_ll_mask(theta, periodogram, K, max_time, fixed_parameter):

    # Fixed parameter must be tuple (index, fixed value)
    dim = (periodogram[0]).shape[0]

    idx, value_parameter = fixed_parameter
        
    theta_mid = value_parameter * np.ones((4,))
    indices = [i!=idx for i in range(4)]
    theta_mid[indices] = theta
        
    #print(theta_aux)

    theta_aux = tuple((np.array([[i]]) for i in theta_mid))
    
    ll = np.sum([grad_spectral_w_mask(theta_aux, j/max_time, periodogram[j-1], np.array([[1.0]])) for j in range(1, K+1)], axis=0)
    ll /= max_time
    print("here",theta, ll[1:][indices])
    
    return (ll[0], ll[1:][indices])

In [11]:
class univariate_estimator_bfgs(object):
    def __init__(self, fixed_parameter, loss=uni_grad_ll_mask, grad=True, initial_guess="random", options=None):
        self.fixed_parameter = fixed_parameter
        self.loss = loss
        self.grad = True # By default uses grad version of spectral ll
        self.initial_guess = initial_guess
        
        if options is None:
            self.options = {'disp': False}#,"maxls":40}
        else:
            self.options = options

    def fit(self, periodogram, max_time):

        np.random.seed(0)

        K = int(periodogram.shape[0])
        self.dim = (periodogram[0]).shape[0]

        # Bounds
        bounds = [(1e-16, None), (1e-16, 1 - 1e-16), (1e-16, None), (1e-16, None)]

        # Initial point
        if isinstance(self.initial_guess, str) and self.initial_guess == "random":
            init_a = np.random.uniform(0, 3, 3)
            init_alpha = np.random.uniform(0, 1, (self.dim, self.dim))
        
            self.init = np.concatenate((init_a[0].ravel(), init_alpha.ravel(), init_a[1:].ravel()))

        # Mask of non-fixed parameters
        indices = [i!=self.fixed_parameter[0] for i in range(4)]
        print(indices)
        bounds = np.array(bounds)[indices]
        self.init = self.init[indices]

        #else:
        #    param_mask = np.array([True]*(self.dim * (2 + self.dim) + 1))
        print(self.init)
        # Estimation
        self.res = minimize(self.loss,
                       self.init, tol=1e-6,
                       method="L-BFGS-B", jac=self.grad,
                       args=(periodogram, K, max_time, self.fixed_parameter),
                       bounds=bounds, options=self.options)

        #theta_estim = np.zeros((self.dim * (2+self.dim) + 1))
    
        #true_indices = np.where(param_mask)[0]
        #theta_estim[true_indices] = self.res.x[:len(true_indices)]

        #self.mu_estim = theta_estim[0: self.dim].reshape((self.dim, 1))
        #self.alpha_estim = theta_estim[self.dim: -self.dim-1].reshape((self.dim, self.dim))
        #self.beta_estim = theta_estim[-self.dim-1:-1].reshape((self.dim, 1))
        #self.noise_estim = theta_estim[-1]

        #return self.mu_estim, self.alpha_estim, self.beta_estim, self.noise_estim
        return self.res.x

# Estimation

In [8]:
mu = np.array([[1.0]])

alpha = np.array([[0.5]])

beta = np.array([[1.0]])

noise = 0.5

theta = np.concatenate((mu.ravel(), alpha.ravel(), beta.ravel(), np.array([noise])))
theta

print("Spectral radius:", np.max(np.abs(np.linalg.eig(alpha)[0])))

Spectral radius: 0.5


In [18]:
it = 10
max_time = 8000

np.random.seed(it)
hp = multivariate_exponential_hawkes(mu, alpha, beta, max_time=max_time)
hp.simulate()
hp_times = hp.timestamps

pp = multivariate_exponential_hawkes(noise * np.ones((1,1)), 0*alpha, beta, max_time=max_time)
pp.simulate()
pp_times = pp.timestamps

idx = np.argsort(pp_times[1:-1] + hp_times, axis=0)[:, 0]
parasited_times = np.array(pp_times[1:-1] + hp_times)[idx]
K = int(parasited_times.shape[0])
#K = int(K*np.log(K))

start_time = time.time()
periodogram = fast_multi_periodogram(K, parasited_times, max_time)
end_time = time.time()
print("In :", end_time - start_time, " s.")

In : 0.0682680606842041  s.


In [21]:
estim = univariate_estimator_bfgs((1, 0.5))
start_time = time.time()
res = estim.fit(periodogram, max_time)
end_time = time.time()
print("In: ", end_time-start_time, " sec.")

[True, False, True, True]
[1.64644051 2.1455681  1.80829013]
here [1.64644051 2.1455681  1.80829013] [0.56150352 0.09228219 0.23058958]
here [1.08493699 2.05328591 1.57770055] [0.52879787 0.06130022 0.21652155]
here [1.0000000e-16 1.9756947e+00 1.0000000e-16] [-1.33157242e+32 -1.15128011e+15 -5.72524741e+31]
here [0.72329133 2.02742217 1.05180037] [0.1315713  0.01904404 0.04538521]
here [0.57999727 2.00747925 0.96894295] [-0.27027391 -0.00468814 -0.1234331 ]
here [0.66915772 2.01988814 1.02049851] [ 0.00983817  0.01077901 -0.00595953]
here [0.63721512 2.00545436 1.09023356] [ 0.00160463  0.00925812 -0.00728758]
here [0.5187005  1.93390662 1.38504546] [-0.00697515  0.00500405 -0.00047854]
here [0.56204615 1.7997765  1.30775839] [0.00300663 0.00577948 0.00234926]
here [1.02441856e+00 1.00000000e-16 4.92433612e-01] [-1.46576339e-01 -3.05869404e-13 -7.32881696e-02]
here [0.57349511 1.75521161 1.28756985] [0.00411038 0.00583036 0.00269117]
here [0.79895684 0.87760581 0.89000173] [-0.0177707

In [20]:
#
res # 165, 0.94033425, 0.95709814, 0.61751876

array([0.4948421 , 0.86929579, 0.50115099])

In [62]:
mask = np.array([[False, True],
                 [True, False]])
estim_red = multivariate_estimator_bfgs(mask=mask)
start_time = time.time()
res_red = estim_red.fit(periodogram, max_time)
end_time = time.time()
print("In: ", end_time-start_time, " sec.")

In:  160.27114534378052  sec.


In [63]:
res_red

  message: ABNORMAL_TERMINATION_IN_LNSRCH
  success: False
   status: 2
      fun: 17.131985185334354
        x: [ 1.531e+00  1.382e+00  3.512e-01  4.120e-01  1.091e+00
             1.534e+00  1.000e-16]
      nit: 55
      jac: [ 5.150e-10  1.164e-09 -5.642e-10  8.100e-09  4.045e-11
             1.396e-09  2.012e-03]
     nfev: 91
     njev: 91
 hess_inv: <7x7 LbfgsInvHessProduct with dtype=float64>

In [7]:
from multiprocess import Pool, cpu_count
# Reduced model estimation (parallel) Good

max_time = 1000
repetitions = 5
# estimations_max = np.zeros((repetitions, 5))


bounds = [(1e-16, None), (1e-16, None), (1e-16, None), 
          (1e-16, None), (1e-16, None)]

dim = 2

def job(it):
    np.random.seed(it)
    hp = multivariate_exponential_hawkes(mu, alpha * beta, beta, max_time=max_time)
    hp.simulate()
    hp_times = hp.timestamps

    pp = multivariate_exponential_hawkes(noise * np.ones((2,1)), 0*alpha, beta, max_time=max_time)
    pp.simulate()
    pp_times = pp.timestamps

    idx = np.argsort(pp_times[1:-1] + hp_times, axis=0)[:, 0]
    parasited_times = np.array(pp_times[1:-1] + hp_times)[idx]
    K = int(parasited_times.shape[0])
    K = int(np.log(K) * K)
    init = np.random.uniform(0, 3, 5)
    
    periodogram = fast_multi_periodogram(K, parasited_times, max_time)
    
    start_time = time.time()
    res = minimize(spectral_multivariate_noised_ll_single,
                   init, tol=1e-16,
                   method="L-BFGS-B", jac=None,
                   args=(periodogram, K, max_time),
                   bounds=bounds, options={"disp":False})
    end_time = time.time()
    
    print('-', end='')
    # print(res.x)

    return res.x
print('|'+'-'*repetitions+'|')
print('|', end='')
with Pool(5) as p:
    estimations_max = np.array(p.map(job, range(repetitions)))
print(estimations_max)
print(estimations_max.mean(axis=0))

|-----|
|-----[[8.04391699e-01 7.07924065e-01 7.70970933e-01 1.16643960e+00
  6.60687520e-01]
 [1.46170525e+00 1.72482212e+00 2.35983817e-01 1.97789173e+00
  1.00000000e-16]
 [3.91540886e-01 5.27518394e-01 1.09868181e+00 1.22473443e+00
  1.05261324e+00]
 [8.68772653e-01 1.00339625e+00 4.77232705e-01 2.23202514e+00
  5.98577835e-01]
 [1.50297040e+00 1.49331815e+00 3.20984519e-01 1.58413529e+00
  1.00000000e-16]]
[1.00587618 1.0913958  0.58077076 1.63704524 0.46237572]


In [32]:
# Full model estimation (parallel) good
max_time = 1000
repetitions = 5
# estimations_max = np.zeros((repetitions, 5))


bounds = [(1e-16, None), (1e-16, None), (1e-16, 1-1e-16), (1e-16, 1-1e-16), (1e-16, 1-1e-16), (1e-16, 1-1e-16), 
          (1e-16, None), (1e-16, None), (1e-16, None)]

dim = 2

def job(it):
    np.random.seed(it)
    hp = multivariate_exponential_hawkes(mu, alpha * beta, beta, max_time=max_time)
    hp.simulate()
    hp_times = hp.timestamps

    pp = multivariate_exponential_hawkes(noise * np.ones((2,1)), 0*alpha, beta, max_time=max_time)
    pp.simulate()
    pp_times = pp.timestamps

    idx = np.argsort(pp_times[1:-1] + hp_times, axis=0)[:, 0]
    parasited_times = np.array(pp_times[1:-1] + hp_times)[idx]
    K = int(parasited_times.shape[0])
    K = int(np.log(K) * K)
    
    init_a = np.random.chisquare(3, dim*2 + 1)

    a = np.random.chisquare(3, (dim, dim))
    radius = np.max(np.abs(np.linalg.eig(a)[0]))
    div = np.random.uniform(1e-16, 1 - 1e-16)
    init_alpha = a * div / (radius)

    init = np.concatenate((init_a[0:2].ravel(), init_alpha.ravel(), init_a[2:].ravel()))
    
    periodogram = fast_multi_periodogram(K, parasited_times, max_time)
    
    res = minimize(grad_ll,
                   init, tol=1e-16,
                   method="L-BFGS-B", jac=True,
                   args=(periodogram, K, max_time),
                   bounds=bounds, options={"disp":False})
    
    print('-', end='')
    # print(res.x)

    return res.x
print('|'+'-'*repetitions+'|')
print('|', end='')
start_time = time.time()
with Pool(5) as p:
    estimations = np.array(p.map(job, range(repetitions)))
print('|\n Done')
end_time = time.time()
print(end_time-start_time)

print(estimations)
print(estimations.mean(axis=0))

|-----|
|-----|
 Done
[[1.41159461e+00 1.29973177e+00 1.00000000e-16 2.68702085e-02
  4.02243830e-01 5.01557528e-02 2.95385583e+00 1.22506070e+00
  1.00000000e-16]
 [1.40050260e+00 1.72473962e+00 1.00000000e-16 2.95677902e-02
  2.36041570e-01 1.00000000e-16 3.61299155e-01 1.97690699e+00
  1.00000000e-16]
 [1.33950023e+00 1.46226425e+00 6.75674759e-02 1.00000000e-16
  2.75144591e-01 7.60037305e-02 4.30800960e+00 1.32612993e+00
  1.00000000e-16]
 [8.69599370e-01 1.00414837e+00 1.00000000e-16 1.00000000e-16
  4.76795689e-01 4.22529772e-05 8.45508432e-01 2.23188091e+00
  5.97752887e-01]
 [1.46935398e+00 1.48755953e+00 3.28564845e-03 1.08483357e-02
  3.22241590e-01 1.00000000e-16 1.32052422e+01 1.57679922e+00
  6.23558479e-03]]
[1.29811016 1.39568871 0.01417062 0.01345727 0.34249345 0.02524035
 4.33478304 1.66735555 0.12079769]


In [33]:
# Full model estimation (parallel) bad
max_time = 1000
repetitions = 5
# estimations_max = np.zeros((repetitions, 5))


bounds = [(1e-16, None), (1e-16, None), (1e-16, 1-1e-16), (1e-16, 1-1e-16), (1e-16, 1-1e-16), (1e-16, 1-1e-16), 
          (1e-16, None), (1e-16, None), (1e-16, None)]

dim = 2

def job(it):
    np.random.seed(it)
    hp = multivariate_exponential_hawkes(mu, alpha * beta, beta, max_time=max_time)
    hp.simulate()
    hp_times = hp.timestamps

    pp = multivariate_exponential_hawkes(noise * np.ones((2,1)), 0*alpha, beta, max_time=max_time)
    pp.simulate()
    pp_times = pp.timestamps

    idx = np.argsort(pp_times[1:-1] + hp_times, axis=0)[:, 0]
    parasited_times = np.array(pp_times[1:-1] + hp_times)[idx]
    K = int(parasited_times.shape[0])
    K = int(np.log(K) * K)
    init = np.random.rand(9)/2 + np.r_[.75, .75, 0., 0., 0., 0., .75, .75, .75]
    
    periodogram = [multivariate_periodogram_bad(j/max_time, parasited_times) for j in range(1, K+1)]
    
    start_time = time.time()
    res = minimize(spectral_multivariate_noised_ll,
                   init, tol=1e-16,
                   method="L-BFGS-B", jac=None,
                   args=(periodogram, K, max_time),
                   bounds=bounds, options={"disp":False})
    end_time = time.time()
    
    print('-', end='')
    # print(res.x)

    return res.x
print('|'+'-'*repetitions+'|')
print('|', end='')
with Pool(5) as p:
    estimations = np.array(p.map(job, range(repetitions)))
print('|\n Done')
print(estimations)
print(estimations.mean(axis=0))

|-----|
|-----|
 Done
[[9.45959446e-01 1.79370733e+00 1.00000000e-16 2.60740057e-01
  2.89011036e-02 7.49201763e-02 1.22102990e+00 3.26424047e+00
  1.00000000e-16]
 [4.41349857e-01 1.40067929e+00 1.00000000e-16 2.48707815e-01
  1.00000000e-16 1.00000000e-16 1.98396551e+00 5.35961373e-01
  6.70504703e-01]
 [7.69808394e-01 1.43343025e+00 7.87493824e-02 1.54809560e-01
  1.00000000e-16 1.53710342e-01 3.11169456e+00 8.27501565e-01
  3.17606196e-01]
 [1.09144610e+00 1.87479322e+00 1.00000000e-16 1.85547704e-01
  1.00000000e-16 7.06698514e-02 2.54439117e+00 1.18621304e+00
  1.00000000e-16]
 [1.04514436e+00 1.79623517e+00 1.00000000e-16 2.30808676e-01
  1.00000000e-16 9.12814046e-02 1.68943754e+00 3.72968375e-01
  1.00000000e-16]]
[0.85874163 1.65976905 0.01574988 0.21612276 0.00578022 0.07811635
 2.11010374 1.23737696 0.19762218]


In [12]:
# Without noise model estimation (parallel)
max_time = 3000
repetitions = 50
# estimations_max = np.zeros((repetitions, 5))


bounds = [(1e-16, None), (1e-16, None), 
          (1e-16, 1-1e-16), (1e-16, 1-1e-16), (1e-16, 1-1e-16), (1e-16, 1-1e-16), 
          (1e-16, None), (1e-16, None)]

dim = 2

def job(it):
    np.random.seed(it)
    hp = multivariate_exponential_hawkes(mu, alpha * beta, beta, max_time=max_time)
    hp.simulate()
    hp_times = hp.timestamps

    pp = multivariate_exponential_hawkes(noise * np.ones((2,1)), 0*alpha, beta, max_time=max_time)
    pp.simulate()
    pp_times = pp.timestamps

    idx = np.argsort(pp_times[1:-1] + hp_times, axis=0)[:, 0]
    parasited_times = np.array(pp_times[1:-1] + hp_times)[idx]
    K = int(parasited_times.shape[0])
    
    #init_a = np.random.uniform(0,3, dim*2)
    #a = np.random.uniform(0,3, (dim,dim))
    #radius = np.max(np.abs(np.linalg.eig(a)[0]))
    #div = np.random.uniform(1e-16, 1 - 1e-16)
    #init_alpha = a * div / (radius)
    
    #init = np.concatenate((init_a[0:dim].ravel(), init_alpha.ravel(), init_a[dim:].ravel()))
    init = np.random.rand(8)/2 + np.r_[.75, .75, 0., 0., 0., 0., .75, .75]
    
    periodogram = [multivariate_periodogram(j/max_time, parasited_times) for j in range(1, K+1)]
    
    start_time = time.time()
    res = minimize(spectral_multivariate_ll,
                   init, tol=1e-16,
                   method="L-BFGS-B", jac=None,
                   args=(periodogram, K, max_time),
                   bounds=bounds, options={"disp":False})
    end_time = time.time()
    
    print('-', end='')
    # print(res.x)

    return res.x
print('|'+'-'*repetitions+'|')
print('|', end='')
with Pool(4) as p:
    estimations_nonoised_model = np.array(p.map(job, range(repetitions)))
print('|\n Done')
#print(estimations)

|--------------------------------------------------|
|

/Users/alisdair/PycharmProjects/hawkes-inhibition/venv/lib/python3.10/site-packages/scipy/optimize/_numdiff.py:598: ComplexWarning: Casting complex values to real discards the imaginary part
  J_transposed[i] = df / dx
/Users/alisdair/PycharmProjects/hawkes-inhibition/venv/lib/python3.10/site-packages/scipy/optimize/_numdiff.py:598: ComplexWarning: Casting complex values to real discards the imaginary part
  J_transposed[i] = df / dx
/Users/alisdair/PycharmProjects/hawkes-inhibition/venv/lib/python3.10/site-packages/scipy/optimize/_lbfgsb_py.py:353: ComplexWarning: Casting complex values to real discards the imaginary part
  _lbfgsb.setulb(m, x, low_bnd, upper_bnd, nbd, f, g, factr,
/Users/alisdair/PycharmProjects/hawkes-inhibition/venv/lib/python3.10/site-packages/scipy/optimize/_lbfgsb_py.py:353: ComplexWarning: Casting complex values to real discards the imaginary part
  _lbfgsb.setulb(m, x, low_bnd, upper_bnd, nbd, f, g, factr,
/Users/alisdair/PycharmProjects/hawkes-inhibition/venv

--------------------------------------------------|
 Done


In [13]:
import pickle
explanation = "Estimations for Scenario (Single)."
explanation += " First is explanation, second is estimations in non-noised model,"
explanation += " Dimensions (repetitions, parameters)."
explanation
toSave = (explanation, estimations_nonoised_model)

In [14]:
with open('estimationsSingleNonNoisedModel2410.pkl', 'wb') as file:
    pickle.dump(toSave, file)